In [1]:
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import pandas as pd

In [14]:
# model = Word2Vec.load("../models/b25-sn-v256/b25-sn-v256.model")
model = Word2Vec.load("../models/b25-sn-v512/b25-sn-v512-a.model")

In [6]:
import pickle

with open('../data/tokenized_data/playlist_names/dataset_test.pkl', 'rb') as f:
    tokenized_playlists = pickle.load(f)

In [15]:
tested = 0
correct = 0

test_set = tokenized_playlists[:250]
a = 0
for playlist in test_set:
    a += len(playlist)
print(f"Total songs: {a}")

for playlist in test_set:
    for song in playlist:
        try:
            tested += 1
            similar_words = model.wv.most_similar(song, topn=1)
            
            if any(word[0] in playlist for word in similar_words):
                correct += 1
        except:
            continue


print(f"Accuracy: {correct/tested}")

Total songs: 10253
Accuracy: 0.5000487662147665


## Unit tested Results

### General 
Testing with 250 Playlist that contain 10k songs all together

#### b25-sn-v50 param: vector_size=50, window=5, min_count=1, sg=0
Accuracy: 0.260704184141227

#### b25-sn-v256-a param: vector_size=256, window=5, min_count=1, sg=0
Accuracy: 0.28089339705452065

#### b25-sn-v256-b param: vector_size=256, window=10, min_count=1, sg=0
Accuracy: 0.3198088364381157

#### b25-sn-v256-c param: vector_size=256, window=20, min_count=1, sg=0
Accuracy: 0.39529893689651807

#### b25-sn-v256-d param: vector_size=256, window=20, min_count=1, sg=1, ns_exponent=0.0
Accuracy: 0.4845411099190481

#### b25-sn-v512-a param: vector_size=512, window=100, min_count=1, sg=0
Accuracy: 0.5000487662147665


## Old not Accurate Results...

In [ ]:
# first model b25-sn-v256: just the base data with no tweaks of the data and the most basic var of the model
# 0.07389817629179331 accuracy over 100 playlists is very low as far as i feel how good the suggested songs are from the model 
# don't know if the number will rise with more playlist but the code needs more optimizing 

# second model b25-sn-v256-b: training it with a bigger dataset - base is the normal dataset but extended with shuffeld playlists on top 
# 0.2773556231003039 accuracy over 100 playlist - thats a solid score if you think about every 1/4 songs that is suggested is correct
# decent result for a second draft

# third model b25-sn-v256-c: based on model the data of b25-sn-v256-b but the context window is now 20 not 5 
# 0.48822188449848025 accuracy over 100 playlists thats a good score for songs suggestion

# b25-sn-v256-d: reduced training data to 80% of the set to have a better testing without the chance of overfitting
# 0.21337249481686246 accuracy over 100 playlists - was to be anticipated because of reducing the training set and the split of train and test data
# still need to test all models against a way bigger chunck of the test set 
# before splitting test and train the data should be shuffeld to remove bias of the data to a specific section 

# tested b25-sn-v256-c again but with only the first pick beeing good or bad
# 0.37232204561161025 is still a solide score for a normal test without mostly any biase

# b25-sn-v256-f bigger vector of 512 and a bigger window of 100 while training
#  accuracy  

# comments about the testing 
# still need to test them against way more playlists not only 100
# also need to cut between training and test data to prevent overfitting the model to the values i just want to here
# it would be interesting what accuracy is possible to reach with this approach
# could also extend the context window 
# also some basic data exploration needs to be done